In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [20]:
# Você pode baixar o conjunto de dados MovieLens 100k em https://grouplens.org/datasets/movielens/
# Carregue os arquivos 'movies.csv' e 'ratings.csv'

movies = pd.read_csv('../data/reduced/movies_m10_rich_pre.csv')
ratings = pd.read_csv('../data/reduced/ratings_m10.csv')

In [21]:
# Junte os conjuntos de dados 'movies' e 'ratings' usando a coluna 'movieId'
data = pd.merge(ratings, movies, on='movieId')

# Divida os dados em conjuntos de treinamento e teste
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)########

In [22]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['genres'])

In [23]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [24]:
user_movie_matrix = train_data.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

In [25]:
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(user_movie_matrix.values.T, cosine_sim)

Ridge()

In [26]:
user_movie_matrix_test = test_data.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
user_movie_matrix_test = user_movie_matrix_test.reindex(columns=user_movie_matrix.columns, fill_value=0)

In [27]:
for _ in range(abs(len(user_movie_matrix.index) - len(user_movie_matrix_test.index))):
    new_row = pd.DataFrame([0 for i in user_movie_matrix_test.columns])
    new_row.index = user_movie_matrix_test.columns
    new_row = new_row.T
    new_row = new_row.iloc[:,:-1]
    user_movie_matrix_test = pd.concat([user_movie_matrix_test, new_row])
user_movie_matrix_test.index = user_movie_matrix.index

In [28]:
user_movie_matrix_test = user_movie_matrix_test.fillna(0)

In [29]:
predicted_ratings = ridge_model.predict(user_movie_matrix_test.values.T)

In [30]:
print(len(predicted_ratings), 'x', len(predicted_ratings[0]))

2026 x 2026


In [31]:
print(len(user_movie_matrix_test.values), 'x', len(user_movie_matrix_test.values.T))

610 x 2026


In [32]:
mse = mean_squared_error(user_movie_matrix_test.values.T, predicted_ratings)
print(f'Mean Squared Error: {mse}')

ValueError: y_true and y_pred have different number of output (610!=2026)